In [ ]:
class NTT(object):
#    
    def __init__(self, n=128):
        if not any([n == t for t in [32,64,128,256,512,1024,2048]]):
            raise ValueError("improper argument ",n)
        self.n = n  
        self.q = 1 + 2*n
        while True:
            if (self.q).is_prime():
                break
            self.q += 2*n
            
        self.F = GF(self.q) ;  self.R = PolynomialRing(self.F, name="w")
        w = (self.R).gen(); self.w = w
        
        g = (w^n + 1)
        xi = g.roots(multiplicities=False)[-1]
        self.xi = xi
        rs = [xi^(2*i+1)  for i in range(n)] 
        self.base = crt_basis([(w - r) for r in rs])  
    
    
    def ntt(self,f):
        def _expand_(f): 
            u = f.list()
            return u + [0]*(self.n-len(u)) 
        
        def _ntt_(xi,N,f):
            if N==1:
                return f
            N_ = N/2 ; xi2 =  xi^2  
            f0 = [f[2*i]   for i in range(N_)] ; f1 = [f[2*i+1] for i in range(N_)] 
            ff0 = _ntt_(xi2,N_,f0) ; ff1 = _ntt_(xi2,N_,f1)  
    
            s  = xi ; ff = [self.F(0) for i in range(N)] 
            for i in range(N_):
                a = ff0[i] ; b = s*ff1[i]  
                ff[i] = a + b ; ff[i + N_] = a - b 
                s = s * xi2                     
            return ff 
        
        return _ntt_(self.xi,self.n,_expand_(f))
        
    def ntt_inv(self,ff):                              ## transformada inversa
        return sum([ff[i]*self.base[i] for i in range(self.n)])
    
    def random_pol(self,args=None):
        return (self.R).random_element(args)

In [19]:
# Parâmetros do NewHope

n = 1024
q = 12289

if n != 2^(log(n,2)) or not is_prime(q) or q < n or q%(2*n) != 1:
    raise ValueError(f"os parâmetros n={n} e q={q} não verificam as condições do esquema NewHope")
    
# noise
k = 8

T = NTT(1024)

In [20]:
## Os anéis usuais com o módulo ciclotómico

R_.<w> = ZZ[]
R.<x>  = QuotientRing(R_,R_.ideal(w^n+1))

Rq_.<w> = GF(q)[]
Rq.<x>  = QuotientRing(Rq_,Rq_.ideal(w^n+1))

In [21]:
# Geradores aleatorios

import random, os

random.seed(os.urandom(8))         # gerar seeds diferentes para cada execução

def random_pol(n=n):
    p = R_.random_element(n)
    return Rq(p)

def chi(k=k):
    samples = range(2*k)
    return sum([random.choice([-1,1]) for _ in samples])


def random_err(n=n):
    return Rq([chi() for _ in range(n)])

In [14]:
## messages

d = GF(q)((q-1)//2)      # o elemento mais afastado de zero

def encode(m):
    return d*Rq(m)

def dq(x,y=0):     # distancia entre elementos de GF(q)
    x_ = lift(x-y)
    return min(x_,q-x_)

def decode(p):      # p está em Rq
    return [0 if dq(x) < dq(x,d) else 1 for x in p.list()]

In [22]:
## KeyGen

# chave privada
e = T.random_pol()
s = T.random_pol()

## chave publica
a = random_pol()
b = a*s + e

TypeError: '<=' not supported between instances of 'NoneType' and 'int'

In [16]:
## Encrypt
m = [random.choice([0,1]) for _ in range(n)]       # mensagem aleatória

r  = random_err()
e0 = random_err()
e1 = random_err()

u = a*r + e0
v = b*r + e1 + encode(m)

In [17]:
## Decrypt

m1 = decode(v - u*s)

In [18]:
m == m1

True